### Azure Cosmos MongoDB 구성
<img 
    src="https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/includes/media/conceptual-object-model/resource-hiearchy.png"
    width="50%"
    height="50%"
/>

[코드 참고 링크](https://github.com/Azure-Samples/azure-cosmos-db-mongodb-python-getting-started)

In [1]:
import pymongo
import os
from dotenv import load_dotenv
load_dotenv()

True

서버 연결 및 닫기

In [2]:
connection_string = os.getenv("MONGODB_CONNECTION_STRING")  # 연결 문자열
account_name = "mongodb01"  # MongoDB 계정 이름 (리소스 이름)

In [3]:
# client
client = pymongo.MongoClient(connection_string)  # 클라이언트 인증

C:\Users\Zenuser\AppData\Local\Temp\ipykernel_7896\2971548342.py:2: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = pymongo.MongoClient(connection_string)  # 클라이언트 인증


In [4]:
# client.close()  # 클라이언트 연결 해제

서버 정보 가져오기

In [5]:
# Get server information
for k, v in client.server_info().items():
    print("Key: {} , Value: {}".format(k, v))

# Get server status of admin database
print("Server status {}".format(client.admin.command("serverStatus")))

# List databases
databases = client.list_database_names()
print("Databases: {}".format(databases))

Key: version , Value: 7.0.0
Key: versionArray , Value: [7, 0, 0, 0]
Key: bits , Value: 64
Key: maxBsonObjectSize , Value: 16777216
Key: ok , Value: 1.0
Server status {'ok': 1.0}
Databases: []


데이터베이스 존재하는지 확인 후, 없으면 새로 만들기

In [6]:
# Get list of databases
databases = client.list_database_names()
if not databases:
    print("No databases found")

# Does database exist?
DB_NAME = "test_db"
if DB_NAME in databases:
    print("Database exists: {}".format(DB_NAME))
else:
    client[DB_NAME].command(
        {"customAction": "CreateDatabase", "offerThroughput": 400}
    )
    print("Created db '{}' with shared throughput.\n".format(DB_NAME))

No databases found
Created db 'test_db' with shared throughput.



컬렉션 존재하는지 확인 후, 없으면 만들기

In [17]:
# Get list of collections
collections = client[DB_NAME].list_collection_names()
if not collections:
    print("No collections found")

# Does database exist?
COLLECTION_NAME = "test_collection"
if COLLECTION_NAME in databases:
    print("Collection exists: {}".format(DB_NAME))
else:
    client[DB_NAME].create_collection(
        name=COLLECTION_NAME, 
        # maxDocuments=1000, shardKey="shardKey"
    )
    print("Created collection '{}' with shared throughput.\n".format(COLLECTION_NAME))

No collections found
Created collection 'test_collection' with shared throughput.



데이터베이스 및 컬렉션 가져오기

In [8]:
database = client.get_database(DB_NAME)  # 데이터베이스 가져오기
collection = database.get_collection(COLLECTION_NAME)  # 컬렉션 가져오기
print(database)
print(collection)

Database(MongoClient(host=['mongodb01.mongo.cosmos.azure.com:10255'], document_class=dict, tz_aware=False, connect=True, replicaset='globaldb', retrywrites=False, maxidletimems=120000, appname='@mongodb01@', tls=True), 'test_db')
Collection(Database(MongoClient(host=['mongodb01.mongo.cosmos.azure.com:10255'], document_class=dict, tz_aware=False, connect=True, replicaset='globaldb', retrywrites=False, maxidletimems=120000, appname='@mongodb01@', tls=True), 'test_db'), 'test_collection')


[문서 만들기](https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/tutorial-insert)

In [ ]:
# 문서 1, 2
new_documents = [
    {
        "_id": "chat-0001",  # _id는 고유값. 추가하지 않으면 자동 생성(ObjectId)
        "username": "user01",
        "user_message": "안녕",
        "ai_massage": "안녕하세요! 😊 오늘 어떻게 도와드릴까요?"
    },
    {
        "_id": "chat-0002",
        "username": "user01",
        "user_message": "넌 누구야?",
        "ai_massage": "저는 OpenAI에서 개발한 AI 도우미입니다! 😊..."
    }
]
insert_result = collection.insert_many(new_documents)
for inserted_id in insert_result.inserted_ids:
    print("Inserted document with id: {}".format(inserted_id))

Inserted document with id: chat-0001
Inserted document with id: chat-0002


In [20]:
# 문서 3
new_document = {
    "_id": "chat-0003",
    "username": "user02",
    "user_message": "안녕",
    "ai_massage": "안녕하세요!"
}
insert = collection.insert_one(new_document)
print("Inserted document with id: {}".format(insert.inserted_id))

Inserted document with id: chat-0003


문서 읽기

In [ ]:
filter_1 = {
    "_id": "chat-0002",
}
existing_document = collection.find_one(filter_1)  # 문서 하나
print(existing_document)

{'_id': 'chat-0002', 'username': 'user01', 'user_message': '넌 누구야?', 'ai_massage': '저는 OpenAI에서 개발한 AI 도우미입니다! 😊...'}


In [ ]:
filter_2 = {
    "username": "user01"
}
matched_documents = collection.find(filter_2)  # 여러 문서
for document in matched_documents:
    print(document)

{'_id': 'chat-0001', 'username': 'user01', 'user_message': '안녕', 'ai_massage': '안녕하세요! 😊 오늘 어떻게 도와드릴까요?'}
{'_id': 'chat-0002', 'username': 'user01', 'user_message': '넌 누구야?', 'ai_massage': '저는 OpenAI에서 개발한 AI 도우미입니다! 😊...'}


[문서 업데이트](https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/tutorial-update)

In [25]:
collection.update_one(
    { "_id": "chat-0003" },
    { "$set": { "username": "user10" } }
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

[문서 삭제](https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/tutorial-delete)

In [26]:
collection.delete_one({ "username": "user10" })

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

데이터베이스 및 컬렉션 문서 수 목록 가져오기

In [27]:
# Get list of databases
databases = client.list_database_names()

# Loop through databases
for db in databases:
    print("Database: {}".format(db))

    # Get list of collections
    collections = client[db].list_collection_names()

    # Loop through collections
    for col in collections:
        print("\tCollection: {}".format(col))

        # Get document count
        doc_count = client[db][col].count_documents({})
        print("\tDocument count: {}".format(doc_count))

Database: test_db
	Collection: test_collection
	Document count: 2


컬렉션 삭제

In [ ]:
if COLLECTION_NAME in client[DB_NAME].list_collection_names():
    print("Dropping collection: {}".format(COLLECTION_NAME))
    client[DB_NAME].drop_collection(COLLECTION_NAME)
else:
    print("Didn't find database: {}".format(COLLECTION_NAME))

Dropping collection: test_collection


데이터베이스 삭제

In [ ]:
if DB_NAME in client.list_database_names():
    print("Dropping database: {}".format(DB_NAME))
    client.drop_database(DB_NAME)
else:
    print("Didn't find database: {}".format(DB_NAME))

Dropping database: test_db
